In [1]:
import pandas as pd
combined_dataset = pd.read_pickle('./combined_dataset_notes.pkl')

In [2]:
combined_dataset.head()

,text,HADM_ID,ADMISSION_DAYS,GENDER,AGE,50801,50802,50803,50804,50805,...,G11,G12,G13,G14,G15,G16,G17,G18,G19,G20
47366,"[chest, pa, lat, clip, clip, number, radiology...",100001,7,0,35,462.0,0.0,24.0,26.0,1.0,...,0,1,0,0,0,0,0,0,0,1
47024,"[pm, liver, gallbladder, u, single, organ, cli...",100003,5,1,60,462.0,-6.0,24.0,17.0,1.0,...,0,1,0,0,0,1,0,0,0,0
9046,"[chest, portable, ap, clip, clip, number, radi...",100006,13,0,49,462.0,0.0,24.0,26.0,1.0,...,0,0,0,0,0,1,0,0,0,1
843,"[pm, chest, pa, lat, clip, clip, number, radio...",100009,5,1,61,462.0,2.0,24.0,28.0,1.0,...,0,0,0,0,0,0,0,1,0,1
43466,"[pm, chest, portable, ap, different, physician...",100010,5,0,54,462.0,0.0,23.0,25.0,1.0,...,0,0,0,0,0,0,0,0,0,0


In [3]:
combined_dataset = combined_dataset.drop('HADM_ID',axis=1)

In [4]:
combined_dataset.columns

Index([          'text', 'ADMISSION_DAYS',         'GENDER',            'AGE',
                  50801,            50802,            50803,            50804,
                  50805,            50806,
       ...
                  'G11',            'G12',            'G13',            'G14',
                  'G15',            'G16',            'G17',            'G18',
                  'G19',            'G20'],
      dtype='object', length=504)

In [5]:
!pip install --upgrade gensim
!pip install -U gensim

Requirement already up-to-date: gensim in /home/akshara/anaconda3/lib/python3.7/site-packages (3.8.1)
Requirement already up-to-date: gensim in /home/akshara/anaconda3/lib/python3.7/site-packages (3.8.1)


In [6]:
#import fasttext
from gensim.models import Word2Vec


In [7]:
#pass text here..
model = Word2Vec(combined_dataset.text,size=1500, window=10, min_count=3, workers=50)#play around with param
#model = FastText(combined_dataset.text,size=1500,window=10, min_count=3, workers=50,sg=1)
# summarize the loaded model
#print(model)
# summarize vocabulary
#words = list(model.wv.vocab)
#print(words)
# access vector for one word
#print(model['sentence'])
# save model
model.save('model1500-10-3.bin')
# load model
#new_model = Word2Vec.load('model1500-10-3.bin')
print(model)

Word2Vec(vocab=32438, size=1500, alpha=0.025)


In [8]:
y = combined_dataset.iloc[:,484:]

In [9]:
y.columns

Index(['G1', 'G2', 'G3', 'G4', 'G5', 'G6', 'G7', 'G8', 'G9', 'G10', 'G11',
       'G12', 'G13', 'G14', 'G15', 'G16', 'G17', 'G18', 'G19', 'G20'],
      dtype='object')

In [10]:

x = combined_dataset.iloc[:,:484]
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.15,random_state=15)

In [11]:
x.columns

Index([          'text', 'ADMISSION_DAYS',         'GENDER',            'AGE',
                  50801,            50802,            50803,            50804,
                  50805,            50806,
       ...
                  51512,            51513,            51514,            51515,
                  51516,            51517,            51518,            51519,
                  51529,            51532],
      dtype='object', length=484)

In [12]:
'''%matplotlib inline
from sklearn.decomposition import PCA
from matplotlib import pyplot
X = model[model.wv.vocab]
pca = PCA(n_components=2)
result = pca.fit_transform(X)
# create a scatter plot of the projection
pyplot.scatter(result[:, 0], result[:, 1])
words = list(model.wv.vocab)
for i, word in enumerate(words):
    pyplot.annotate(word, xy=(result[i, 0], result[i, 1]))
pyplot.show()'''

'%matplotlib inline\nfrom sklearn.decomposition import PCA\nfrom matplotlib import pyplot\nX = model[model.wv.vocab]\npca = PCA(n_components=2)\nresult = pca.fit_transform(X)\n# create a scatter plot of the projection\npyplot.scatter(result[:, 0], result[:, 1])\nwords = list(model.wv.vocab)\nfor i, word in enumerate(words):\n    pyplot.annotate(word, xy=(result[i, 0], result[i, 1]))\npyplot.show()'

In [13]:
import numpy as np
WordVectorz=dict(zip(model.wv.index2word,model.wv.vectors))
class AverageEmbeddingVectorizer(object):
    def __init__(self, word2vec):
        self.word2vec = word2vec
        # if a text is empty we should return a vector of zeros
        # with the same dimensionality as all the other vectors
        self.dim =100 # because we use 100 embedding points 

    def fit(self, X, y):
        return self

    def transform(self, X):
        return np.array([
            np.mean([self.word2vec[w] for w in words if w in self.word2vec]
                    or [np.zeros(self.dim)], axis=0)
            for words in X
        ])
AverageEmbeddingVectorizer(WordVectorz).fit(x_train.text,y_train)
x_train.text = AverageEmbeddingVectorizer(WordVectorz).transform(x_train.text)

/home/akshara/anaconda3/envs/mimic/lib/python3.7/site-packages/pandas/core/generic.py:5208: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


In [14]:
x_test.text = AverageEmbeddingVectorizer(WordVectorz).transform(x_test.text)

In [15]:
x_train.head()

,text,ADMISSION_DAYS,GENDER,AGE,50801,50802,50803,50804,50805,50806,...,51512,51513,51514,51515,51516,51517,51518,51519,51529,51532
14968,-0.315506,3,1,74,462.0,-2.0,24.0,22.0,1.0,104.0,...,1.0,1.044,1.0,1.0,5.0,3.0,0.0,0.0,135.5,43.0
34477,0.033534,16,0,47,462.0,4.0,24.0,32.0,1.0,104.0,...,1.0,1.044,1.0,1.0,12.0,3.0,0.0,0.0,135.5,43.0
17630,-0.171170,5,0,45,462.0,0.0,24.0,26.0,1.0,104.0,...,1.0,1.044,1.0,1.0,0.0,3.0,0.0,0.0,135.5,43.0
44811,0.007256,3,0,55,462.0,2.0,24.0,32.0,1.0,104.0,...,1.0,1.044,1.0,1.0,61.0,3.0,0.0,0.0,135.5,43.0
6700,0.021348,11,1,49,365.0,0.0,24.0,29.0,1.0,109.0,...,1.0,1.044,1.0,1.0,2.0,3.0,0.0,0.0,135.5,43.0


In [16]:
x_train.shape

(38697, 484)

In [17]:
x_train.to_pickle('./train-bothwhadm.pkl')
x_test.to_pickle('./test-bothwhadm.pkl')
y_train.to_pickle('./y_train-bothwhadm.pkl')
y_test.to_pickle('./y_test-bothwhadm.pkl')

In [18]:
from sklearn.pipeline import Pipeline
from sklearn.multiclass import OneVsRestClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
from sklearn.multiclass import OneVsRestClassifier
from sklearn.multioutput import ClassifierChain
from sklearn.model_selection import GridSearchCV,StratifiedKFold
from sklearn.metrics import jaccard_score,roc_auc_score,confusion_matrix,hamming_loss

clf_multilabel = OneVsRestClassifier(LGBMClassifier(iterations=1000,learning_rate=0.15,n_jobs=-1))

#from sklearn.feature_extraction.text import TfidfVectorizer

In [19]:

clf_multilabel.fit(x_train,y_train)


OneVsRestClassifier(estimator=LGBMClassifier(boosting_type='gbdt',
                                             class_weight=None,
                                             colsample_bytree=1.0,
                                             importance_type='split',
                                             iterations=1000,
                                             learning_rate=0.15, max_depth=-1,
                                             min_child_samples=20,
                                             min_child_weight=0.001,
                                             min_split_gain=0.0,
                                             n_estimators=100, n_jobs=-1,
                                             num_leaves=31, objective=None,
                                             random_state=None, reg_alpha=0.0,
                                             reg_lambda=0.0, silent=True,
                                             subsample=1.0,
                           

In [20]:
predicted1=clf_multilabel.predict(x_test)

In [21]:
predicted1_prob_train = clf_multilabel.predict_proba(x_train)
predicted1_prob_test = clf_multilabel.predict_proba(x_test)

In [22]:
print(jaccard_score(y_test,predicted1,average='micro'))
print(jaccard_score(y_test,predicted1,average='macro'))
print(roc_auc_score(y_test,predicted1))
print(hamming_loss(y_test,predicted1))


0.5059825633831045
0.37989694002712404
0.6664987463717846
0.18047298286718408


In [23]:
predicted1

array([[0, 0, 0, ..., 0, 0, 1],
       [0, 0, 1, ..., 0, 0, 1],
       [0, 0, 1, ..., 1, 0, 1],
       ...,
       [0, 0, 1, ..., 1, 0, 1],
       [0, 0, 1, ..., 1, 0, 1],
       [0, 0, 0, ..., 1, 1, 0]])

In [24]:
#pipe1.fit(x_train.text,y_train)
predicted1_prob_testdf = pd.DataFrame(predicted1_prob_test)
predicted1_prob_traindf = pd.DataFrame(predicted1_prob_train)

In [25]:
predicted1_prob_test.shape

(6829, 20)

In [26]:
predicted1_prob_testdf

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,0.002308,0.007166,0.023274,0.011296,0.004303,0.023268,0.055258,0.016996,0.052532,0.007305,0.000000e+00,0.061223,0.007568,0.181553,9.998923e-01,0.072856,0.001993,0.012108,0.005044,0.991483
1,0.021374,0.065149,0.830165,0.276001,0.155486,0.516682,0.897741,0.401665,0.688078,0.938288,0.000000e+00,0.198199,0.147756,0.013485,2.537059e-06,0.474976,0.012762,0.423572,0.093800,0.636720
2,0.072169,0.255100,0.717782,0.585935,0.320398,0.509312,0.814466,0.574090,0.645601,0.325479,0.000000e+00,0.249939,0.407995,0.014373,2.351297e-06,0.569131,0.018108,0.693504,0.301143,0.705197
3,0.002891,0.032008,0.534295,0.052111,0.080799,0.061786,0.998782,0.191719,0.148939,0.331984,0.000000e+00,0.015671,0.056689,0.003744,1.789256e-06,0.108649,0.008223,0.268517,0.066568,0.357987
4,0.010331,0.073621,0.937778,0.146605,0.198116,0.189178,0.970115,0.199451,0.245045,0.234819,0.000000e+00,0.046227,0.082531,0.063797,1.143742e-05,0.163417,0.005675,0.259245,0.207954,0.772583
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6824,0.040075,0.078302,0.715778,0.592924,0.167042,0.525066,0.829588,0.928699,0.710419,0.424111,0.000000e+00,0.330084,0.130377,0.016281,1.969582e-06,0.645929,0.018158,0.830335,0.193598,0.352700
6825,0.022308,0.049433,0.583670,0.051255,0.242133,0.529720,0.980423,0.679418,0.130552,0.146966,4.905412e-276,0.020490,0.121951,0.094484,6.622385e-06,0.274351,0.006315,0.286398,0.179521,0.303874
6826,0.040434,0.141840,0.883184,0.875093,0.104737,0.711946,0.943064,0.307649,0.752078,0.867112,0.000000e+00,0.259349,0.103245,0.024741,3.123076e-05,0.718607,0.029118,0.729105,0.348973,0.854684
6827,0.024660,0.149521,0.761720,0.534467,0.158748,0.216167,0.990286,0.369929,0.458424,0.638416,0.000000e+00,0.079979,0.132604,0.063103,8.644109e-07,0.245992,0.003250,0.555613,0.154110,0.588828


In [27]:

predicted1_prob_testdf.to_pickle('./pred-test-bothwhadm.pkl')
predicted1_prob_traindf.to_pickle('./pred-train-bothwhadm.pkl')

In [25]:
import numpy as np

a=np.random.rand(4,3)
b=np.random.rand(4,3)
print(a)
print(b)

[[0.74054759 0.36312163 0.1753963 ]
 [0.21834233 0.30399263 0.8736924 ]
 [0.30154406 0.98975319 0.61806732]
 [0.23620411 0.79692109 0.29076069]]
[[0.34349274 0.01216964 0.79399147]
 [0.8032902  0.51174569 0.30523294]
 [0.2960148  0.98963371 0.26255512]
 [0.92346791 0.54923816 0.46663071]]


In [27]:
c=np.stack((a,b))
c

array([[[0.74054759, 0.36312163, 0.1753963 ],
        [0.21834233, 0.30399263, 0.8736924 ],
        [0.30154406, 0.98975319, 0.61806732],
        [0.23620411, 0.79692109, 0.29076069]],

       [[0.34349274, 0.01216964, 0.79399147],
        [0.8032902 , 0.51174569, 0.30523294],
        [0.2960148 , 0.98963371, 0.26255512],
        [0.92346791, 0.54923816, 0.46663071]]])